In [1]:
import os
import sys 
import yaml
import math
import time
import random
import traceback
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
import scipy.sparse
from scipy.ndimage import gaussian_filter

from tqdm.auto import tqdm

import numpy.fft as FFT
import tensorflow as tf
from typing import List, Dict

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from tensorflow.keras.layers import (Input, Conv2D, Dense, Flatten, 
                                     MaxPool2D, RepeatVector, Lambda,
                                     LeakyReLU, Dropout)
from tensorflow.keras.models import Model, save_model
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow.keras.backend as K

from keras_radam import RAdam
from keras_radam.training import RAdamOptimizer

sys.path.append("/glade/work/schreck/repos/holodec-ml")
from library.losses import SymmetricCrossEntropy
from library.callbacks import get_callbacks

from multiprocessing import cpu_count, Pool

Using TensorFlow backend.


### Load the configuration

In [2]:
config_file = "notebook.yml"

In [3]:
with open(config_file) as config_file:
    config = yaml.load(config_file, Loader=yaml.FullLoader)

In [4]:
try:
    os.makedirs(config["path_save"])
except:
    pass

### Set up some globals

In [5]:
path_data = config["path_data"]
num_particles = config["num_particles"]
split = 'train'
subset = False
output_cols = ["x", "y", "z", "d", "hid"]

batch_size = config["conv2d_network"]["batch_size"]

input_shape = (600, 400, 1)

n_particles = config["num_particles"]
output_channels = len(output_cols) - 1

In [ ]:
def OpticsFFT(Ain):
    """
    Apply 2D fft to input matrix
    """
    Aout = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(Ain)))/np.sqrt(np.size(Ain))
    return Aout

def OpticsIFFT(Ain):
    """
    Apply 2D inverse fft to input matrix
    """
    Aout = np.fft.ifftshift(np.fft.ifft2(np.fft.fftshift(Ain)))*np.sqrt(np.size(Ain))
    return Aout

In [6]:
num_particles_dict = {
    1 : '1particle',
    3 : '3particle',
    'multi': 'multiparticle',
    '50-100': '50-100'}

split_dict = {
    'train' : 'training',
    'test'   : 'test',
    'valid': 'validation'}


class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(
        self, 
        path_data: str, 
        num_particles: int, 
        split: str, 
        subset: bool, 
        output_cols: List[str], 
        batch_size: int, 
        shuffle: bool = True,
        maxnum_particles: int = False,
        scaler: Dict[str, str] = False) -> None:
        
        'Initialization'
        self.ds = self.open_dataset(path_data, num_particles, split)
        self.batch_size = batch_size
        self.output_cols = [x for x in output_cols if x != 'hid']        
        self.subset = subset
        self.hologram_numbers = self.ds.hologram_number.values
        if shuffle:
            random.shuffle(self.hologram_numbers)
        self.num_particles = num_particles
        self.xsize = len(self.ds.xsize.values)
        self.ysize = len(self.ds.ysize.values)
        self.shuffle = shuffle
        self.maxnum_particles = maxnum_particles
                
        if not scaler:
            self.scaler = {col: StandardScaler() for col in output_cols}
            for col in output_cols:
                scale = self.ds[col].values
                self.scaler[col].fit(scale.reshape(scale.shape[-1], -1))
        else:
            self.scaler = scaler
            
#         self.images = np.vstack(self.ds["image"].values for col in output_cols).T
#         self.columns = np.vstack(self.ds[col].values for col in output_cols).T
#         self.df = zip(self.images, self.columns)
        
    def get_transform(self):
        return self.scaler

    def __len__(self):
        'Denotes the number of batches per epoch'
        return math.ceil(len(self.hologram_numbers) / self.batch_size)
    
    def __getitem__(self, idx):
        'Generate one batch of data'
        holograms = self.hologram_numbers[
            idx * self.batch_size: (idx + 1) * self.batch_size
        ]
        x_out, y_out = self._batch(holograms)
        return x_out, y_out
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            random.shuffle(self.hologram_numbers)
            
    def _batch(self, holograms):
        'Create a batch of data'
        try:
        
            x_out = np.zeros((
                len(holograms), self.xsize, self.ysize
            ))
            y_out = np.zeros((
                len(holograms), 
                self.maxnum_particles if self.maxnum_particles else self.num_particles, 
                len(self.output_cols)
            ))
            a = time.time()
            for k, hologram in enumerate(holograms):
                im = self.ds["image"][hologram].values
                x_out[k] = (im-np.mean(im)) / (np.std(im))
                #A = np.log(np.abs(OpticsFFT(A)))                    
                particles = np.where(self.ds["hid"] == hologram + 1)[0]  
                for l, p in enumerate(particles):
                    for m, col in enumerate(self.output_cols):
                        val = self.ds[col][p].values
                        y_out[k, l, m] = self.scaler[col].transform(
                            val.reshape(1, -1)
                        )
                if self.maxnum_particles and len(particles) < self.maxnum_particles:
                    for l in range(len(particles), self.maxnum_particles):
                        for m, col in enumerate(self.output_cols):
                            val = y_out[k, l, m]
                            y_out[k, l, m] = self.scaler[col].transform(
                                val.reshape(1, -1)
                            )
            #
            # convert y_out to sparse if we are using padding
#             if self.maxnum_particles:
#                 y_out = sparse_vstack([
#                     csr_matrix(y_out[i]) for i in y_out.shape[0]
#                 ])
            
            x_out = np.expand_dims(x_out, axis=-1)
            return x_out, y_out
        
        except:
            print(traceback.print_exc())
    
    def open_dataset(self, path_data, num_particles, split):
        """
        Opens a HOLODEC file

        Args: 
            path_data: (str) Path to dataset directory
            num_particles: (int or str) Number of particles per hologram
            split: (str) Dataset split of either 'train', 'valid', or 'test'

        Returns:
            ds: (xarray Dataset) Opened dataset
        """
        path_data = os.path.join(path_data, self.dataset_name(num_particles, split))

        if not os.path.isfile(path_data):
            print(f"Data file does not exist at {path_data}. Exiting.")
            raise 

        ds = xr.open_dataset(path_data)
        return ds
    
    def dataset_name(self, num_particles, split, file_extension='nc'):
        """
        Return the dataset filename given user inputs

        Args: 
            num_particles: (int or str) Number of particles per hologram
            split: (str) Dataset split of either 'train', 'valid', or 'test'
            file_extension: (str) Dataset file extension

        Returns:
            ds_name: (str) Dataset name
        """

        valid = [1,3,'multi','50-100']
        if num_particles not in valid:
            raise ValueError("results: num_particles must be one of %r." % valid)
        num_particles = num_particles_dict[num_particles]

        valid = ['train','test','valid']
        if split not in valid:
            raise ValueError("results: split must be one of %r." % valid)
        split = split_dict[split]
        ds_name = f'synthetic_holograms_{num_particles}_{split}.{file_extension}'

        return ds_name

In [7]:
train_gen = DataGenerator(
    path_data, num_particles, "train", subset, 
    output_cols, batch_size, maxnum_particles = 3
)

In [8]:
# scaler = {col: StandardScaler() for col in train_gen.output_cols}
# for col in train_gen.output_cols:
#     scale = train_gen.ds[col].values
#     scaler[col].fit(scale.reshape(scale.shape[-1], -1))
#     result = scaler[col].transform(scale.reshape(scale.shape[-1], -1))
#     print(col, min(result), max(result), np.mean(result), np.std(result))

In [9]:
train_scalers = train_gen.get_transform()

In [10]:
valid_gen = DataGenerator(
    path_data, num_particles, "test", subset, 
    output_cols, batch_size, scaler = train_scalers, maxnum_particles = 3
)

### Initialize callbacks

In [11]:
callbacks = get_callbacks(config)

### Set up a model

##### Custom losses

In [12]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def R2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return SS_res/(SS_tot + K.epsilon())

def keras_mse(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))

In [13]:
# def wsme(y_true, y_pred):
#     cond1 = K.equal(y_true, 0.0)
#     zero = K.switch(cond1,K.square(y_pred - y_true), 
#                              K.zeros_like(y_true))
#     cond2 = K.greater(y_true, 0.0)
#     real = K.switch(cond2, 
#                              K.square(y_pred - y_true), 
#                              K.zeros_like(y_true))
#     w1 = K.sum(K.cast(cond1, "float"))
#     w2 = K.sum(K.cast(cond2, "float"))
#     total = w1 + w2
#     zero = K.sum(zero) / w1
#     real = K.sum(real) / w2
#     return (real + zero)

def wsme(y_true, y_pred):
    
    w = K.abs(K.mean(y_true[1]))
    w = w / (1 - w)
    
    # w = K.sum(K.cast(K.greater(y_true[1], 0), "float")) # Number actually not zero
    
    error = K.square(y_true - y_pred)
    error = K.switch(K.equal(y_true, 0), w * error, error)
    return error

In [14]:
n_particles = 3

custom_losses = {
    "sce": SymmetricCrossEntropy(1.0, 1.0),
    "weighted_mse": wsme,
    "r2": R2,
    "rmse": rmse
}

class Conv2DNeuralNetwork(object):
    """
    A Conv2D Neural Network Model that can support an arbitrary numbers of
    layers.

    Attributes:
        filters: List of number of filters in each Conv2D layer
        kernel_sizes: List of kernel sizes in each Conv2D layer
        conv2d_activation: Type of activation function for conv2d layers
        pool_sizes: List of Max Pool sizes
        dense_sizes: Sizes of dense layers
        dense_activation: Type of activation function for dense layers
        output_activation: Type of activation function for output layer
        lr: Optimizer learning rate
        optimizer: Name of optimizer or optimizer object.
        adam_beta_1: Exponential decay rate for the first moment estimates
        adam_beta_2: Exponential decay rate for the first moment estimates
        sgd_momentum: Stochastic Gradient Descent momentum
        decay: Optimizer decay
        loss: Name of loss function or loss object
        batch_size: Number of examples per batch
        epochs: Number of epochs to train
        verbose: Level of detail to provide during training
        model: Keras Model object
    """
    def __init__(
        self, 
        filters=(8,), 
        kernel_sizes=(5,),
        conv2d_activation="relu", 
        pool_sizes=(4,), 
        pool_dropout=0.0,
        dense_sizes=(64,),
        dense_activation="relu", 
        dense_dropout = 0.0,
        output_activation="linear",
        lr=0.001, 
        optimizer="adam", 
        adam_beta_1=0.9,
        adam_beta_2=0.999, 
        sgd_momentum=0.9, 
        decay=0, 
        loss="mse",
        metrics = [], 
        batch_size=32, 
        epochs=2, 
        verbose=0
    ):
        
        self.filters = filters
        self.kernel_sizes = [tuple((v,v)) for v in kernel_sizes]
        self.conv2d_activation = conv2d_activation
        self.pool_sizes = [tuple((v,v)) for v in pool_sizes]
        self.pool_dropout = pool_dropout
        self.dense_sizes = dense_sizes
        self.dense_activation = dense_activation
        self.dense_dropout = dense_dropout
        self.output_activation = output_activation
        self.lr = lr
        self.optimizer = optimizer
        self.optimizer_obj = None
        self.adam_beta_1 = adam_beta_1
        self.adam_beta_2 = adam_beta_2
        self.sgd_momentum = sgd_momentum
        self.decay = decay
        self.loss = custom_losses[loss] if loss in custom_losses else loss
        self.metrics = []
        for m in metrics:
            if m in custom_losses:
                self.metrics.append(custom_losses[m])
            else:
                self.metrics.append(m)
        self.batch_size = batch_size
        self.epochs = epochs
        self.verbose = verbose
        self.model = None
        
        if self.conv2d_activation == "leakyrelu":
            self.conv2d_activation = LeakyReLU(alpha=0.1)
        if self.dense_activation == "leakyrelu":
            self.dense_activation = LeakyReLU(alpha=0.1)
        if self.output_activation == "leakyrelu":
            self.output_activation = LeakyReLU(alpha=0.1)

    def build_neural_network(self, input_shape, n_particles, output_shape):
        """Create Keras neural network model and compile it."""
        
        # Input
        conv_input = Input(shape=(input_shape), name="input")
        
        # ConvNet encoder
        nn_model = conv_input
        for h in range(len(self.filters)):
            nn_model = Conv2D(self.filters[h],
                              self.kernel_sizes[h],
                              padding="same",
                              activation=self.conv2d_activation,
                              kernel_initializer='he_uniform',
                              name=f"conv2D_{h:02d}")(nn_model)
            nn_model = MaxPool2D(self.pool_sizes[h],
                                 name=f"maxpool2D_{h:02d}")(nn_model)
            nn_model = Dropout(self.pool_dropout, 
                               name = f"maxpool2D_dr_{h:02d}")(nn_model)
        nn_model = Flatten()(nn_model)
        
        # Classifier
        for h in range(len(self.dense_sizes)):
            nn_model = Dense(self.dense_sizes[h],
                             activation=self.dense_activation,
                             kernel_initializer='he_uniform',
                             name=f"dense_{h:02d}")(nn_model)
            nn_model = Dropout(self.dense_dropout, 
                               name=f"dense_dr_{h:02d}")(nn_model)
        
        # Output
        nn_model = RepeatVector(n_particles, name = "repeat")(nn_model)
        nn_model = Dense(output_shape,
                         activation=self.output_activation,
                         name=f"dense_output")(nn_model)
        nn_model = Lambda(
            self.LastLayer,
            input_shape = (n_particles, output_shape)
        )(nn_model)
        
        self.model = Model(conv_input, nn_model)
        
        if self.optimizer == "adam":
            self.optimizer_obj = Adam(lr=self.lr, clipnorm = 1.0)
        elif self.optimizer == "sgd":
            self.optimizer_obj = SGD(lr=self.lr, momentum=self.sgd_momentum,
                                     decay=self.decay)
        elif self.optimizer == "radam":
            self.optimizer_obj = RAdam(total_steps=10000, warmup_proportion=0.1, min_lr=1e-7)
            
        self.model.compile(
            optimizer=self.optimizer_obj, 
            loss=self.loss,
            metrics=self.metrics
        )
        self.model.summary()

    def fit(self, x, y, xv=None, yv=None, callbacks=None):
        
        if len(x.shape[1:])==2:
            x = np.expand_dims(x, axis=-1)
        if len(y.shape) == 1:
            output_shape = 1
        else:
            output_shape = y.shape[1]
        
        input_shape = x.shape[1:]
        self.build_neural_network(input_shape, output_shape)
        self.model.fit(x, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose, validation_data=(xv, yv), callbacks=callbacks)
        return self.model.history.history
    
    def LastLayer(self, x):
        return 1.75 * K.tanh(x / 100) 

    def predict(self, x):
        y_out = self.model.predict(np.expand_dims(x, axis=-1),
                                   batch_size=self.batch_size)
        return y_out

    def predict_proba(self, x):
        y_prob = self.model.predict(x, batch_size=self.batch_size)
        return y_prob
    
    def load_weights(self, weights):
        try:
            self.model.load_weights(weights)
            self.model.compile(
                optimizer=self.optimizer, 
                loss=self.loss, 
                metrics=self.metrics
            )
        except:
            print("You must first call build_neural_network before loading weights. Exiting.")
            sys.exit(1)

In [15]:
mod = Conv2DNeuralNetwork(**config["conv2d_network"])

In [16]:
mod.build_neural_network(input_shape, n_particles, output_channels)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 600, 400, 1)]     0         
_________________________________________________________________
conv2D_00 (Conv2D)           (None, 600, 400, 8)       208       
_________________________________________________________________
maxpool2D_00 (MaxPooling2D)  (None, 120, 80, 8)        0         
_________________________________________________________________
maxpool2D_dr_00 (Dropout)    (None, 120, 80, 8)        0         
_________________________________________________________________
conv2D_01 (Conv2D)           (None, 120, 80, 12)       2412      
_________________________________________________________________
maxpool2D_01 (MaxPooling2D)  (None, 24, 16, 12)        0         
_________________________________________________________________
maxpool2D_dr_01 (Dropout)    (None, 24, 16, 12)        0     

### Train a model

In [17]:
mod.model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    epochs=config["conv2d_network"]["epochs"],
    verbose=True,
    callbacks=callbacks,
    use_multiprocessing=True,
    workers=16
)

Epoch 1/100
391/391 [==============================] - 339s 866ms/step - loss: 1.6867 - mae: 1.0296 - R2: 1.0235 - val_loss: 1.7235 - val_mae: 0.9881 - val_R2: 1.0454
Epoch 2/100
391/391 [==============================] - 316s 809ms/step - loss: 1.4390 - mae: 0.9613 - R2: 0.8735 - val_loss: 1.4666 - val_mae: 0.9290 - val_R2: 0.8898
Epoch 3/100
391/391 [==============================] - 314s 804ms/step - loss: 1.3447 - mae: 0.9259 - R2: 0.8161 - val_loss: 1.3171 - val_mae: 0.8924 - val_R2: 0.7992
Epoch 4/100
391/391 [==============================] - 318s 814ms/step - loss: 1.3029 - mae: 0.9070 - R2: 0.7907 - val_loss: 1.2763 - val_mae: 0.8823 - val_R2: 0.7745
Epoch 5/100
391/391 [==============================] - 317s 812ms/step - loss: 1.2749 - mae: 0.8943 - R2: 0.7737 - val_loss: 1.2574 - val_mae: 0.8775 - val_R2: 0.7630
Epoch 6/100
391/391 [==============================] - 320s 819ms/step - loss: 1.2527 - mae: 0.8842 - R2: 0.7604 - val_loss: 1.2469 - val_mae: 0.8758 - val_R2: 0.756

In [18]:
# with Pool(12) as p:
#     x_train, y_train = [], []
#     _total = train_gen.__len__()
#     for (x, y) in tqdm(p.imap(train_gen.__getitem__, range(_total)), total = _total):
#         x_train.append(x)
#         y_train.append(y)
# x_train = np.vstack(x_train)
# y_train = np.vstack(y_train)

In [19]:
# with Pool(12) as p:
#     x_valid, y_valid = [], []
#     _total = valid_gen.__len__()
#     for (x, y) in tqdm(p.imap(valid_gen.__getitem__, range(_total)), total = _total):
#         x_valid.append(x)
#         y_valid.append(y)
# x_valid = np.vstack(x_valid)
# y_valid = np.vstack(y_valid)

In [20]:
# mod.model.fit(
#     x_train, y_train,
#     validation_data=(x_valid, y_valid),
#     epochs=config["conv2d_network"]["epochs"],
#     verbose=True,
#     callbacks=callbacks,
#     shuffle=True,
#     batch_size=batch_size
# )

In [21]:
def fitness(learning_rate, num_dense_layers, 
            num_input_nodes, num_dense_nodes, 
            activation, batch_size, adam_decay):
    
    # Reload generators
    
    # Update the configuration
    
    # Load the model
    model = 
    
    # train
    blackbox = model.model.fit_generator(
        generator=train_gen,
        validation_data=valid_gen,
        epochs=config["conv2d_network"]["epochs"],
        verbose=True,
        callbacks=callbacks,
        use_multiprocessing=True,
        workers=16
    )
    
    #return the validation accuracy for the last epoch.
    objective = blackbox.history['val_loss'][-1]

    # Print the classification accuracy.
    print()
    print("Accuracy: {0:.2%}".format(accuracy))
    print()


    # Delete the Keras model with these hyper-parameters from memory.
    del model
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    tf.reset_default_graph()
    
    # the optimizer aims for the lowest score, so we return our negative accuracy
    return objective

AttributeError: module 'tensorflow' has no attribute 'reset_default_graph'